In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs 
import requests

In [2]:
url_hoteles = "https://ibis.accor.com/es/destination/city/hotels-madrid-v2418.html"

In [3]:
respuesta_hoteles = requests.get(url_hoteles)
respuesta_hoteles.status_code

200

In [4]:
sopa_hoteles = bs(respuesta_hoteles.content, "html.parser")

In [31]:
hoteles = sopa_hoteles.find_all("li", class_="aem-GridColumn aem-GridColumn--default--3")[:10]
len(hoteles)

10

In [32]:
dictio_scrap = {
    "nombre_hotel": [],
    "descripcion": [],
    "valoracion": []}

In [33]:
for hotel in hoteles:
    # Nombre del hotel (Ejemplo: en un <h2>, <h3> o <a>)
    nombre = hotel.find("h3")
    nombre_hotel = nombre.get_text(strip=True) if nombre else "Sin nombre"

    # Descripción (Ejemplo: en un <p>) 
    descripcion = hotel.find("p")
    descripcion_texto = descripcion.get_text(strip=True) if descripcion else "Sin descripción"

    # Valoración (Ejemplo: en un <span> con una clase específica)
    valoracion = hotel.find("span", class_="cmp-teaser__rating-score")  # Buscar la clase correcta
    valoracion_texto = valoracion.get_text(strip=True) if valoracion else "Sin valoración"
    if valoracion_texto != "Sin valoración":
        valoracion_texto = valoracion_texto.split("/")[0]  # Tomar solo el número antes de "/"
        valoracion_texto = valoracion_texto.replace(",", ".")  # Convertir "4,7" a "4.7"
        valoracion_texto = float(valoracion_texto)  # Convertir a número flotante

    # Agregar los datos al diccionario
    dictio_scrap["nombre_hotel"].append(nombre_hotel)
    dictio_scrap["descripcion"].append(descripcion_texto)
    dictio_scrap["valoracion"].append(valoracion_texto)

In [35]:
df_hoteles = pd.DataFrame(dictio_scrap)
df_hoteles

,nombre_hotel,descripcion,valoracion
0,ibis Styles Madrid Prado,"Estamos en el centro de Madrid, con una ubicac...",4.7
1,ibis budget Madrid Centro Lavapies,Nuestra ubicación te permitirá descubrir por t...,4.3
2,ibis Styles Madrid Centro Maravillas,ibis Styles Madrid Centro Maravillas es un hot...,4.7
3,ibis budget Madrid Calle 30,"Tú, si tú! buscas un hotel ibis budget Madrid ...",4.3
4,ibis Madrid Centro las Ventas,El hotel ibis Madrid Centro las Ventas tiene u...,4.5
5,ibis budget Madrid Centro las Ventas,Ibis budget Madrid Centro está situado a 15 mi...,4.3
6,ibis Styles Madrid City Las Ventas,Junto a la principal arteria madrileña está u...,4.6
7,ibis budget Madrid Calle Alcalá,El ibis budget Madrid Calle Alcalá es el punto...,4.3
8,ibis Madrid Calle Alcalá,"Bienvenido al hotel ibis Madrid Calle Alcalá, ...",4.5
9,ibis budget Madrid Vallecas,ibis budget Madrid Vallecas es un hotel modern...,4.2


In [28]:
df_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nombre_hotel  126 non-null    object
 1   descripcion   126 non-null    object
 2   valoracion    126 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


In [48]:
def scrap_info_hoteles(url):
    dictio_scrap = {
        "nombre_hotel": [],
        "descripcion": [],
        "valoracion": []
    }

    res_hoteles = requests.get(url)

    if res_hoteles.status_code == 200:
        sopa_hoteles = bs(res_hoteles.content, "html.parser")

        # Buscar los hoteles y manejar caso en que la lista esté vacía
        hoteles = sopa_hoteles.find_all("li", class_="aem-GridColumn aem-GridColumn--default--3")
        hoteles = hoteles[:9] if hoteles else []  # Evita errores de slicing si está vacío

        for hotel in hoteles:
            # Extraer nombre del hotel
            nombre = hotel.find("h3")
            nombre_hotel = nombre.get_text(strip=True) if nombre else "Sin nombre"

            # Extraer descripción
            descripcion = hotel.find("p")
            descripcion_texto = descripcion.get_text(strip=True) if descripcion else "Sin descripción"

            # Extraer valoración
            valoracion = hotel.find("span", class_="cmp-teaser__rating-score")
            valoracion_texto = valoracion.get_text(strip=True) if valoracion else "Sin valoración"

            # Limpiar y convertir la valoración a float
            if valoracion_texto != "Sin valoración":
                valoracion_texto = valoracion_texto.split("/")[0]  # Tomar solo el número antes de "/"
                valoracion_texto = valoracion_texto.replace(",", ".")  # Convertir "4,7" a "4.7"
                try:
                    valoracion_texto = float(valoracion_texto)
                except ValueError:
                    valoracion_texto = "Error en conversión"

            # Agregar datos al diccionario
            dictio_scrap["nombre_hotel"].append(nombre_hotel)
            dictio_scrap["descripcion"].append(descripcion_texto)
            dictio_scrap["valoracion"].append(valoracion_texto)

    return dictio_scrap

In [50]:
dictio_final = scrap_info_hoteles(url_hoteles)
df = pd.DataFrame(dictio_final)
df

,nombre_hotel,descripcion,valoracion
0,ibis Styles Madrid Prado,"Estamos en el centro de Madrid, con una ubicac...",4.7
1,ibis budget Madrid Centro Lavapies,Nuestra ubicación te permitirá descubrir por t...,4.3
2,ibis Styles Madrid Centro Maravillas,ibis Styles Madrid Centro Maravillas es un hot...,4.7
3,ibis budget Madrid Calle 30,"Tú, si tú! buscas un hotel ibis budget Madrid ...",4.3
4,ibis Madrid Centro las Ventas,El hotel ibis Madrid Centro las Ventas tiene u...,4.5
5,ibis budget Madrid Centro las Ventas,Ibis budget Madrid Centro está situado a 15 mi...,4.3
6,ibis Styles Madrid City Las Ventas,Junto a la principal arteria madrileña está u...,4.6
7,ibis budget Madrid Calle Alcalá,El ibis budget Madrid Calle Alcalá es el punto...,4.3
8,ibis Madrid Calle Alcalá,"Bienvenido al hotel ibis Madrid Calle Alcalá, ...",4.5


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre_hotel  9 non-null      object 
 1   descripcion   9 non-null      object 
 2   valoracion    9 non-null      float64
dtypes: float64(1), object(2)
memory usage: 348.0+ bytes


In [52]:
df.to_csv("../data/10_hoteles_competencia.csv", index=False)